In [38]:
import numpy as np
#from ric_fun import classify_rna_data
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [39]:
def classify_rna_data(input_rna, dis, freq, binsize, discard):
    data = input_rna.copy()
    distance = data['E'] - data['S']
    rna_long = data[distance >= dis]
    rna_long_hf = rna_long[rna_long['value'] > freq]
    
    mask_c = (rna_long_hf['S'] <= discard[0]) & (rna_long_hf['value']>discard[1])
    
    hist_data = []
    s = 0
    e = binsize
    lens = data['E'].max()
    while True:
        distance_hf = rna_long_hf[~mask_c]['E'] - rna_long_hf[~mask_c]['S']
        mask = (distance_hf > s) & (distance_hf <= e)
        hist_data.append(rna_long_hf[~mask_c]['value'][mask].sum())
        s = e
        e += binsize
        if e >= lens:
            break
    
    return rna_long_hf[mask_c], rna_long_hf[~mask_c], hist_data

In [44]:
#file_dir = 'C:/Users/10915/Desktop/zwen/matrix-final/'
file_dir = 'C:/Users/10915/Desktop/zwen/matrix-test/'
file_list = os.listdir(file_dir)
counts_array = np.zeros(len(file_list), dtype=[('Name','U14'),('C','<f8'),('NC','<f8')])
nc_classify = np.zeros(len(file_list), dtype=[('Name','U14'),('short','<f8'),('mid','<f8'),('long','<f8')])
df_sum = np.zeros(len(file_list), dtype=[('Name','U50'),('C','<i4'),('NC','<i4'),('short','<i4'),('mid','<i4'),('long','<i4')])
for i,file_name in enumerate(file_list):
    recom_data = np.loadtxt(file_dir +  file_name, usecols=[0,1,2], dtype=[('S','<i4'),('E','<i4'),('value','<f8')])
    min_value = min(2000, recom_data['value'].max() / 100)
    rna_c, rna_nc, hist_data = classify_rna_data(recom_data, 100, 10, 100, [100, min_value])
    total_c = rna_c['value'].sum()
    total_nc = rna_nc['value'].sum()
    total = total_c + total_nc
    counts_array[i][0] = file_name.split('_')[2] #.split('-')[0]
    counts_array[i][1] = total_c / total
    counts_array[i][2] = total_nc / total
    nc_classify[i][0] = counts_array[i][0]
    nc_classify[i][1] = sum(hist_data[0:50]) / total_nc
    nc_classify[i][2] = sum(hist_data[50:200]) / total_nc
    nc_classify[i][3] = sum(hist_data[200:]) / total_nc
    df_sum[i][0] = '_'.join([file_name.split('_')[1], '-'.join(file_name.split('_')[2].split('-')[:-1]), file_name.split('_')[-1].split('.')[0]])
    df_sum[i][1] = total_c 
    df_sum[i][2] = total_nc 
    df_sum[i][3] = sum(hist_data[0:50]) 
    df_sum[i][4] = sum(hist_data[50:200]) 
    df_sum[i][5] = sum(hist_data[200:]) 
    print(df_sum)

#df = pd.DataFrame(df_sum, columns=['Name', 'C', 'NC', 'short', 'mid', 'long'])

# 保存为制表符分隔的文本文件
#output_file_path = "C:/Users/10915/Desktop/zwen/summary.txt"
#df.to_csv(output_file_path, sep='\t', index=False)



[('beta_PHEV-PRJNA1032947-N2a_PHEV-PRJNA1032947-N2a-N', 134659, 61545, 53349, 307, 7889)]
